Step 1, Import Data:

In [1]:
import scala.io.Source
val ocreCex = "https://raw.githubusercontent.com/neelsmith/nomisma/master/cex/ocre-cite-ids.cex"

import scala.io.Source

ocreCex: String = "https://raw.githubusercontent.com/neelsmith/nomisma/master/cex/ocre-cite-ids.cex"

In [3]:
val lines = Source.fromURL(ocreCex).getLines.toVector

lines: Vector[String] = Vector(
  "ID#Label#Denomination#Metal#Authority#Mint#Region#ObvType#ObvLegend#ObvPortraitId#RevType#RevLegend#RevPortraitId#StartDate#EndDate",
  "3.com.43#RIC III Commodus 43#denarius#ar#commodus#rome#italy#Head of Commodus, laureate, right#M COMMODVS ANTONINVS AVG#http://nomisma.org/id/commodus#Roma, helmeted, draped, standing left, holding Victory in extended right hand and vertical spear in left hand#TR P VII IMP V COS III P P#http://collection.britishmuseum.org/id/person-institution/60208#182#182",
  "9.thes.27B.iii#RIC IX Thessalonica 27B: Subtype iii#ae3#ae#valentinian_i#thessalonica#macedonia#Bust of Valens, pearl-diademed, draped and cuirassed, right#D N VALEN-S P F AVG#http://nomisma.org/id/valens#Victory advancing left, holding wreath and palm#SECVRITAS-REIPVBLICAE#http://collection.britishmuseum.org/id/person-institution/60915#367#375",
  "9.thes.27B.ii#RIC IX Thessalonica 27B: Subtype ii#ae3#ae#valentinian_i#thessalonica#macedonia#Bust of Valens, p

Step 2, Extract Needed Data (Authority and Reverse Legend)

*Step 2a: ID Authorities Within Threshold Number of Coins Minted (800):*

In [4]:
val data = lines.tail.map(ln => ln.split("#").toVector)

data: Vector[Vector[String]] = Vector(
  Vector(
    "3.com.43",
    "RIC III Commodus 43",
    "denarius",
    "ar",
    "commodus",
    "rome",
    "italy",
    "Head of Commodus, laureate, right",
    "M COMMODVS ANTONINVS AVG",
    "http://nomisma.org/id/commodus",
    "Roma, helmeted, draped, standing left, holding Victory in extended right hand and vertical spear in left hand",
    "TR P VII IMP V COS III P P",
    "http://collection.britishmuseum.org/id/person-institution/60208",
    "182",
    "182"
  ),
  Vector(
    "9.thes.27B.iii",
    "RIC IX Thessalonica 27B: Subtype iii",
    "ae3",
    "ae",
    "valentinian_i",
    "thessalonica",
    "macedonia",
    "Bust of Valens, pearl-diademed, draped and cuirassed, right",
    "D N VALEN-S P F AVG",
    "http://nomisma.org/id/valens",
    "Victory advancing left, holding wreath and palm",
    "SECVRITAS-REIPVBLICAE",
    "http://collection.britishmuseum.org/id/person-institution/60915",
    "367",
    "375"
  ),
  Vector(
    "9

In [5]:
val Authorities = data.map(columns => columns(4))
val AuthoritiesGrouped = Authorities.groupBy(auth => auth)
val AuthoritiesCounts = AuthoritiesGrouped.map{ case (a, v) => (a, v.size) }
val AuthoritiesHisto = AuthoritiesCounts.toVector.sortBy(frequency => frequency._2).reverse

Authorities: Vector[String] = Vector(
  "commodus",
  "valentinian_i",
  "valentinian_i",
  "severus_alexander",
  "valentinian_i",
  "constantine_i",
  "constantine_i",
  "constantine_i",
  "constantine_i",
  "constantine_i",
  "constantine_i",
  "constantine_i",
  "valerian",
  "constantine_i",
  "constantine_i",
  "constantine_i",
  "constantine_i",
  "trajan_decius",
  "constantine_i",
  "constantine_i",
  "constantine_i",
  "constantine_i",
  "constantine_i",
  "maxentius",
  "maxentius",
  "maxentius",
  "maxentius",
  "maxentius",
  "maxentius",
  "maxentius",
  "maxentius",
  "maxentius",
  "maxentius",
  "maxentius",
  "maxentius",
  "maxentius",
  "maxentius",
  "maxentius",
...
AuthoritiesGrouped: Map[String, Vector[String]] = Map(
  "quietus" -> Vector(
    "quietus",
    "quietus",
    "quietus",
    "quietus",
    "quietus",
    "quietus",
    "quietus",
    "quietus",
    "quietus",
    "quietus",
    "quietus",
    "quietus",
    "quietus"
  ),
  "zenobia" -> Vector("ze

In [6]:
val cutOff = 1000 
val OneThousandClub = AuthoritiesHisto.filter(frequency => frequency._2 > cutOff)

cutOff: Int = 1000
OneThousandClub: Vector[(String, Int)] = Vector(
  ("constantine_i", 4096),
  ("gallienus", 3434),
  ("hadrian", 2434),
  ("antoninus_pius", 2381),
  ("maximian", 2326),
  ("septimius_severus", 2170),
  ("marcus_aurelius", 1819),
  ("constantius_ii", 1617),
  ("valerian", 1597),
  ("vespasian", 1588),
  ("valentinian_i", 1417),
  ("theodosius_i", 1176),
  ("carausius", 1146)
)

*Step 2b: Pair Reverse Legend to Corresponding Authority:*

In [7]:
val ReverseLegend = data.map(columns => columns(10))
val AuthAndLegend = data.map(columns => columns(4) + " " + columns(10))

//then parse (to the best I can)
val AandLParsed3 = AuthAndLegend.map(ln => ln.replaceAll(", ", " "))
val AandLParsed2 = AandLParsed3.map(ln => ln.replaceAll(";" , ""))
val AandLParsed = AandLParsed2.map(ln => ln.replaceAll(":" , ""))

ReverseLegend: Vector[String] = Vector(
  "Roma, helmeted, draped, standing left, holding Victory in extended right hand and vertical spear in left hand",
  "Victory advancing left, holding wreath and palm",
  "Victory advancing left, holding wreath and palm",
  "Pax, draped, standing left, holding olive branch in right hand and sceptre in left hand",
  "Victory advancing left, holding wreath and palm",
  "Sol, chlamys hanging behind, standing left, raising right hand and holding globe close to body in left hand",
  "Sol, chlamys hanging behind, standing left, raising right hand and holding globe close to body in left hand",
  "Sol, chlamys hanging behind, standing left, raising right hand and holding up globe in left hand",
  "Sol, chlamys hanging behind, standing left, raising right hand and holding up globe in left hand",
  "Sol, chlamys hanging behind, standing left, raising right hand and holding up globe in left hand",
  "Roma, helmeted, draped, seated front, head left, seated fr

In [10]:
val LParen = "\\("
val RParen = "\\)"

LParen: String = "\\("
RParen: String = "\\)"

In [12]:
val AandLParsedMore = AandLParsed.map(ln => ln.replaceAll((LParen) , ""))    
val AandLParsedMore2 = AandLParsedMore.map(ln => ln.replaceAll((RParen) , ""))   
/// Also having issue with lowercase vs uppercase and want each word to only appear once in each

AandLParsedMore: Vector[String] = Vector(
  "commodus Roma helmeted draped standing left holding Victory in extended right hand and vertical spear in left hand",
  "valentinian_i Victory advancing left holding wreath and palm",
  "valentinian_i Victory advancing left holding wreath and palm",
  "severus_alexander Pax draped standing left holding olive branch in right hand and sceptre in left hand",
  "valentinian_i Victory advancing left holding wreath and palm",
  "constantine_i Sol chlamys hanging behind standing left raising right hand and holding globe close to body in left hand",
  "constantine_i Sol chlamys hanging behind standing left raising right hand and holding globe close to body in left hand",
  "constantine_i Sol chlamys hanging behind standing left raising right hand and holding up globe in left hand",
  "constantine_i Sol chlamys hanging behind standing left raising right hand and holding up globe in left hand",
  "constantine_i Sol chlamys hanging behind standing left 

In [13]:
val AandLParsedMore3 = AandLParsedMore2.map(a => a.toLowerCase)

AandLParsedMore3: Vector[String] = Vector(
  "commodus roma helmeted draped standing left holding victory in extended right hand and vertical spear in left hand",
  "valentinian_i victory advancing left holding wreath and palm",
  "valentinian_i victory advancing left holding wreath and palm",
  "severus_alexander pax draped standing left holding olive branch in right hand and sceptre in left hand",
  "valentinian_i victory advancing left holding wreath and palm",
  "constantine_i sol chlamys hanging behind standing left raising right hand and holding globe close to body in left hand",
  "constantine_i sol chlamys hanging behind standing left raising right hand and holding globe close to body in left hand",
  "constantine_i sol chlamys hanging behind standing left raising right hand and holding up globe in left hand",
  "constantine_i sol chlamys hanging behind standing left raising right hand and holding up globe in left hand",
  "constantine_i sol chlamys hanging behind standing left

In [14]:
val AuthHead = AandLParsedMore3.map(ln => ln.split(" ").toVector)

AuthHead: Vector[Vector[String]] = Vector(
  Vector(
    "commodus",
    "roma",
    "helmeted",
    "draped",
    "standing",
    "left",
    "holding",
    "victory",
    "in",
    "extended",
    "right",
    "hand",
    "and",
    "vertical",
    "spear",
    "in",
    "left",
    "hand"
  ),
  Vector(
    "valentinian_i",
    "victory",
    "advancing",
    "left",
    "holding",
    "wreath",
    "and",
    "palm"
  ),
  Vector(
    "valentinian_i",
    "victory",
    "advancing",
    "left",
    "holding",
    "wreath",
    "and",
...

Part 3, Analyze Each Member of the 1000 club:

*Constantine_i*

In [15]:
val constantine_iGrouped = AuthHead.filter(_.contains("constantine_i"))

constantine_iGrouped: Vector[Vector[String]] = Vector(
  Vector(
    "constantine_i",
    "sol",
    "chlamys",
    "hanging",
    "behind",
    "standing",
    "left",
    "raising",
    "right",
    "hand",
    "and",
    "holding",
    "globe",
    "close",
    "to",
    "body",
    "in",
    "left",
    "hand"
  ),
  Vector(
    "constantine_i",
    "sol",
    "chlamys",
    "hanging",
    "behind",
    "standing",
    "left",
    "raising",
    "right",
    "hand",
    "and",
    "holding",
    "globe",
    "close",
    "to",
    "body",
...

In [16]:
val constantine_iSumVector = constantine_iGrouped.flatten

constantine_iSumVector: Vector[String] = Vector(
  "constantine_i",
  "sol",
  "chlamys",
  "hanging",
  "behind",
  "standing",
  "left",
  "raising",
  "right",
  "hand",
  "and",
  "holding",
  "globe",
  "close",
  "to",
  "body",
  "in",
  "left",
  "hand",
  "constantine_i",
  "sol",
  "chlamys",
  "hanging",
  "behind",
  "standing",
  "left",
  "raising",
  "right",
  "hand",
  "and",
  "holding",
  "globe",
  "close",
  "to",
  "body",
  "in",
  "left",
  "hand",
...

In [17]:
val constantine_iFF = constantine_iSumVector.filter(!_.contains("constantine_i")).filter(!_.contains("extending")).filter(!_.contains("left")).filter(!_.contains("right")).filter(!_.contains("and")).filter(!_.contains("on")).filter(!_.contains("to")).filter(!_.contains("of"))
.filter(!_.contains("with")).filter(!_.contains("up")).filter(!_.contains("him")).filter(!_.contains("over")).filter(!_.contains("in")).filter(!_.contains("up")).filter(!_.contains("for"))
.filter(!_.endsWith("ed")).filter(!_.endsWith("ing"))
.filter(!_.contains("each")).filter(!_.contains("other")).filter(!_.contains("by")).filter(!_.contains("VOT"))
.filter(!_.contains("Two")).filter(!_.contains("two")).filter(!_.contains("above")).filter(!_.contains("across"))

constantine_iFF: Vector[String] = Vector(
  "sol",
  "chlamys",
  "globe",
  "close",
  "body",
  "sol",
  "chlamys",
  "globe",
  "close",
  "body",
  "sol",
  "chlamys",
  "globe",
  "sol",
  "chlamys",
  "globe",
  "sol",
  "chlamys",
  "globe",
  "roma",
  "head",
  "hexastyle",
  "temple",
  "globe",
  "sceptre",
  "shield",
  "pediment",
  "wreath",
  "mars",
  "spear",
  "or",
  "sceptre",
  "shield",
  "mars",
  "head",
  "spear",
  "or",
  "sceptre",
...

In [18]:
val constantine_iRegrouped = constantine_iFF.groupBy(word => word)

constantine_iRegrouped: Map[String, Vector[String]] = Map(
  "beneath" -> Vector(
    "beneath",
    "beneath",
    "beneath",
    "beneath",
    "beneath",
    "beneath",
    "beneath",
    "beneath",
    "beneath"
  ),
  "liquid" -> Vector(
    "liquid",
    "liquid",
    "liquid",
    "liquid",
    "liquid",
    "liquid",
    "liquid",
    "liquid"
  ),
  "sceptres" -> Vector(
    "sceptres",
    "sceptres",
    "sceptres",
    "sceptres",
    "sceptres",
    "sceptres"
  ),
  "anchises" -> Vector("anchises"),
  "wreaths" -> Vector("wreaths", "wreaths", "wreaths", "wreaths"),
  "salus" -> Vector(
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
...

In [19]:
val constantine_iCount = constantine_iRegrouped.map{ case (a, v) => (a, v.size) } 

constantine_iCount: Map[String, Int] = Map(
  "beneath" -> 9,
  "liquid" -> 8,
  "sceptres" -> 6,
  "anchises" -> 1,
  "wreaths" -> 4,
  "salus" -> 10,
  "down" -> 4,
  "nimbate" -> 14,
  "side" -> 89,
  "bow" -> 23,
  "" -> 3,
  "number" -> 170,
  "felicitas" -> 10,
  "wreath" -> 537,
  "phoenix" -> 1,
  "vota" -> 5,
  "soldier" -> 30,
  "modius" -> 80,
  "x" -> 28,
  "outer" -> 358,
  "dais" -> 1,
  "transverse" -> 206,
  "tempora" -> 3,
  "children" -> 51,
  "city" -> 1,
  "cornucopiae" -> 228,
  "stages" -> 1,
  "body" -> 34,
  "spread" -> 141,
  "palm-tree" -> 1,
  "layers" -> 233,
  "crook" -> 1,
  "vot/xx/mvlt/\u2022xxx\u2022" -> 9,
  "is" -> 5,
  "cippus" -> 19,
  "bust" -> 19,
  "his" -> 5,
  "spears" -> 3,
...

In [20]:
val constantine_i = constantine_iCount.toVector.sortBy(frequency => frequency._2).reverse

constantine_i: Vector[(String, Int)] = Vector(
  ("globe", 1175),
  ("shield", 1027),
  ("spear", 879),
  ("chlamys", 738),
  ("sol", 652),
  ("shoulder", 651),
  ("a", 636),
  ("wreath", 537),
  ("altar", 470),
  ("radiate", 438),
  ("head", 428),
  ("between", 411),
  ("them", 378),
  ("soldiers", 371),
  ("outer", 358),
  ("stars", 354),
  ("three", 317),
  ("laurel", 312),
  ("sceptre", 301),
  ("star", 299),
  ("i", 294),
  ("doors", 276),
  ("camp", 271),
  ("turrets", 267),
  ("gate", 267),
  ("layers", 233),
  ("genius", 228),
  ("cornucopiae", 228),
  ("vot/xx", 222),
  ("trophy", 221),
  ("nude", 217),
  ("transverse", 206),
  ("mars", 205),
  ("patera", 192),
  ("captives", 191),
  ("vot/is/xx", 190),
  ("captive", 178),
  ("vot/pr", 172),
...

In [ ]:
val cutOff2 = 50 
val constantine_iterms = constantine_i.filter(frequency => frequency._2 > cutOff2)

*Gallienus*

In [21]:
val gallienusGrouped = AuthHead.filter(_.contains("gallienus"))
val gallienusSumVector = gallienusGrouped.flatten

gallienusGrouped: Vector[Vector[String]] = Vector(
  Vector(
    "gallienus",
    "securitas",
    "draped",
    "standing",
    "left",
    "leaning",
    "on",
    "column",
    "holding",
    "globe",
    "in",
    "right",
    "hand",
    "and",
    "sceptre",
    "in",
    "left",
    "hand"
  ),
  Vector(
    "gallienus",
    "pax",
    "draped",
    "standing",
    "left",
    "holding",
    "spear",
    "in",
    "right",
    "hand",
    "and",
    "shield",
    "in",
    "left",
    "hand"
  ),
  Vector(
...
gallienusSumVector: Vector[String] = Vector(
  "gallienus",
  "securitas",
  "draped",
  "standing",
  "left",
  "leaning",
  "on",
  "column",
  "holding",
  "globe",
  "in",
  "right",
  "hand",
  "and",
  "sceptre",
  "in",
  "left",
  "hand",
  "gallienus",
  "pax",
  "draped",
  "standing",
  "left",
  "holding",
  "spear",
  "in",
  "right",
  "hand",
  "and",
  "shield",
  "in",
  "left",
  "hand",
  "gallienus",
  "pax",
  "draped",
  "walking",
  "left",
...

In [22]:
val gallienusFF = gallienusSumVector.filter(!_.contains("gallienus")).filter(!_.contains("extending")).filter(!_.contains("left")).filter(!_.contains("right")).filter(!_.contains("and")).filter(!_.contains("on")).filter(!_.contains("to"))
.filter(!_.contains("with")).filter(!_.contains("up")).filter(!_.contains("him")).filter(!_.contains("over")).filter(!_.contains("in")).filter(!_.contains("up")).filter(!_.contains("for"))
.filter(!_.endsWith("ed")).filter(!_.endsWith("ing"))
.filter(!_.contains("each")).filter(!_.contains("other")).filter(!_.contains("by")).filter(!_.contains("VOT"))
.filter(!_.contains("Two")).filter(!_.contains("two")).filter(!_.contains("above")).filter(!_.contains("across"))

gallienusFF: Vector[String] = Vector(
  "securitas",
  "column",
  "globe",
  "sceptre",
  "pax",
  "spear",
  "shield",
  "pax",
  "olive-branch",
  "sceptre",
  "pax",
  "olive-branch",
  "sceptre",
  "pax",
  "olive-branch",
  "sceptre",
  "pax",
  "olive-branch",
  "sceptre",
  "sol",
  "radiate",
  "nude",
  "except",
  "cloak",
  "shoulders",
  "whip",
  "sol",
  "radiate",
  "nude",
  "except",
  "cloak",
  "shoulders",
  "whip",
  "sol",
  "radiate",
  "nude",
  "except",
  "cloak",
...

In [23]:
val gallienusRegrouped = gallienusFF.groupBy(word => word)
val gallienusCount = gallienusRegrouped.map{ case (a, v) => (a, v.size) } 

gallienusRegrouped: Map[String, Vector[String]] = Map(
  "beneath" -> Vector("beneath", "beneath"),
  "diadem" -> Vector("diadem", "diadem", "diadem", "diadem", "diadem"),
  "salus" -> Vector(
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
    "salus",
...
gallienusCount: Map[String, Int] = Map(
  "beneath" -> 2,
  "diadem" -> 5,
  "salus" -> 67,
  "down" -> 16,
  "side" -> 1,
  "anchor" -> 95,
  "bow" -> 26,
  "peacock" -> 6,
  "" -> 6,
  "hippocamp" -> 20,
  "felicitas" -> 75,
  "wreath" -> 386,
  "savus" -> 4,
  "providentia" -> 243,
  "laetitia" -> 87,
  "wild" -> 2,
  "serapis" -> 2,
  "rock" 

In [24]:
val gallienus = gallienusCount.toVector.sortBy(frequency => frequency._2).reverse

gallienus: Vector[(String, Int)] = Vector(
  ("cornucopiae", 887),
  ("sceptre", 803),
  ("globe", 554),
  ("spear", 546),
  ("shield", 465),
  ("sometimes", 450),
  ("at", 440),
  ("or", 409),
  ("feet", 388),
  ("wreath", 386),
  ("patera", 362),
  ("altar", 332),
  ("olive-branch", 257),
  ("providentia", 243),
  ("palm", 238),
  ("nude", 237),
  ("ensign", 232),
  ("her", 227),
  ("genius", 224),
  ("valerian", 223),
  ("military", 221),
  ("attire", 221),
  ("mars", 214),
  ("except", 196),
  ("cloak", 196),
  ("rudder", 185),
  ("his", 167),
  ("thunderbolt", 161),
  ("pax", 153),
  ("radiate", 132),
  ("scales", 130),
  ("sol", 122),
  ("of", 118),
  ("aequitas", 111),
  ("caduceus", 110),
  ("shoulders", 109),
  ("soldier", 107),
  ("anchor", 95),
...

*Hadrian*

In [25]:
val hadrianGrouped = AuthHead.filter(_.contains("hadrian"))
val hadrianSumVector = hadrianGrouped.flatten

hadrianGrouped: Vector[Vector[String]] = Vector(
  Vector(
    "hadrian",
    "securitas",
    "draped",
    "seated",
    "right",
    "on",
    "chair",
    "with",
    "cornucopiae",
    "for",
    "arms",
    "resting",
    "head",
    "on",
    "right",
    "hand"
  ),
  Vector(
    "hadrian",
    "trajan",
    "and",
    "hadrian",
    "both",
    "laureate",
    "and",
    "togate",
    "facing",
    "one",
    "another",
    "clasping",
    "right",
    "hands",
    "and",
    "holding",
    "rolls",
    "in",
    "their",
...
hadrianSumVector: Vector[String] = Vector(
  "hadrian",
  "securitas",
  "draped",
  "seated",
  "right",
  "on",
  "chair",
  "with",
  "cornucopiae",
  "for",
  "arms",
  "resting",
  "head",
  "on",
  "right",
  "hand",
  "hadrian",
  "trajan",
  "and",
  "hadrian",
  "both",
  "laureate",
  "and",
  "togate",
  "facing",
  "one",
  "another",
  "clasping",
  "right",
  "hands",
  "and",
  "holding",
  "rolls",
  "in",
  "their",
  "left",
  "hands",
 

In [26]:
val hadrianFF = hadrianSumVector.filter(!_.contains("hadrian")).filter(!_.contains("extending")).filter(!_.contains("left")).filter(!_.contains("right")).filter(!_.contains("and")).filter(!_.contains("on")).filter(!_.contains("to")).filter(!_.contains("of"))
.filter(!_.contains("with")).filter(!_.contains("up")).filter(!_.contains("him")).filter(!_.contains("over")).filter(!_.contains("in")).filter(!_.contains("up")).filter(!_.contains("for"))
.filter(!_.endsWith("ed")).filter(!_.endsWith("ing")).filter(!_.contains("Hadrian"))
.filter(!_.contains("each")).filter(!_.contains("other")).filter(!_.contains("by")).filter(!_.contains("VOT"))
.filter(!_.contains("Two")).filter(!_.contains("two")).filter(!_.contains("above")).filter(!_.contains("across"))

hadrianFF: Vector[String] = Vector(
  "securitas",
  "chair",
  "cornucopiae",
  "arms",
  "head",
  "trajan",
  "both",
  "laureate",
  "rolls",
  "their",
  "trajan",
  "bust",
  "laureate",
  "shoulder",
  "the",
  "genius",
  "the",
  "roman",
  "people",
  "altar",
  "center",
  "thunderbolt",
  "globe",
  "eagle",
  "center",
  "roma",
  "military",
  "dress",
  "spear",
  "who",
  "her",
  "roma",
  "military",
  "dress",
  "spear",
  "who",
  "her",
  "roma",
...

In [27]:
val hadrianRegrouped = hadrianFF.groupBy(word => word)
val hadrianCount = hadrianRegrouped.map{ case (a, v) => (a, v.size) } 

hadrianRegrouped: Map[String, Vector[String]] = Map(
  "looks" -> Vector(
    "looks",
    "looks",
    "looks",
    "looks",
    "looks",
    "looks",
    "looks",
    "looks",
    "looks",
    "looks"
  ),
  "rostra" -> Vector("rostra", "rostra", "rostra", "rostra", "rostra"),
  "liquid" -> Vector("liquid"),
  "sword" -> Vector(
    "sword",
    "sword",
    "sword",
    "sword",
    "sword",
    "sword",
    "sword",
    "sword",
    "sword",
    "sword",
    "sword",
    "sword",
    "sword",
    "sword",
    "sword",
    "sword",
    "sword",
    "sword",
    "sword",
    "sword",
    "sword",
    "sword",
    "sword",
...
hadrianCount: Map[String, Int] = Map(
  "looks" -> 10,
  "rostra" -> 5,
  "liquid" -> 1,
  "sword" -> 41,
  "tridens" -> 2,
  "matidia" -> 1,
  "rowers" -> 61,
  "octastyle" -> 6,
  "salus" -> 71,
  "tympanum" -> 3,
  "down" -> 14,
  "side" -> 69,
  "anchor" -> 4,
  "bow" -> 23,
  "peacock" -> 2,
  "rod" -> 18,
  "pileus" -> 12,
  "" -> 42,
  "felicitas" -> 104,

In [28]:
val hadrian = hadrianCount.toVector.sortBy(frequency => frequency._2).reverse

hadrian: Vector[(String, Int)] = Vector(
  ("cornucopiae", 586),
  ("patera", 371),
  ("sceptre", 356),
  ("altar", 322),
  ("vertical", 309),
  ("roll", 282),
  ("spear", 254),
  ("arm", 227),
  ("corn-ears", 210),
  ("head", 207),
  ("center", 165),
  ("rudder", 160),
  ("the", 159),
  ("globe", 144),
  ("branch", 143),
  ("at", 138),
  ("raise", 133),
  ("roma", 132),
  ("dress", 130),
  ("her", 127),
  ("out", 117),
  ("military", 105),
  ("felicitas", 104),
  ("shield", 103),
  ("a", 101),
  ("foot", 100),
  ("next", 97),
  ("basket", 96),
  ("caduceus", 96),
  ("pietas", 95),
  ("spes", 93),
  ("rock", 92),
  ("cloak", 84),
  ("low", 83),
  ("round", 77),
  ("bull", 75),
  ("palm", 75),
  ("galley", 75),
...

This is just my experimentation with how easy it would be to apply the same code to another topic within the data set

In [ ]:
val Denomination = data.map(columns => columns(2))
val AuthAndDenom = data.map(columns => columns(4) + " " + columns(2))

In [ ]:
val AuthHead2 = AuthAndDenom.map(ln => ln.split(" ").toVector)

In [ ]:
val gallienusGrouped2 = AuthHead2.filter(_.contains("gallienus"))
val gallienusSumVector2 = gallienusGrouped2.flatten
val gallienusRegrouped2 = gallienusSumVector2.groupBy(word => word)
val gallienusCount2 = gallienusRegrouped2.map{ case (a, v) => (a, v.size) } 

In [ ]:
val gallienus2 = gallienusCount2.toVector.sortBy(frequency => frequency._2).reverse

In [ ]:
val hadrianGrouped2 = AuthHead2.filter(_.contains("hadrian"))
val hadrianSumVector2 = hadrianGrouped2.flatten
val hadrianRegrouped2 = hadrianSumVector2.groupBy(word => word)
val hadrianCount2 = hadrianRegrouped2.map{ case (a, v) => (a, v.size) } 

In [ ]:
val hadrian2 = hadrianCount2.toVector.sortBy(frequency => frequency._2).reverse